In [97]:
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


In [98]:
import tensorflow.keras.models

In [99]:
import numpy as np
import pandas as pd
import os
import PIL
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import shutil
from sklearn.metrics import confusion_matrix, classification_report

In [100]:
train_df = pd.read_csv('capstone covid data/train.txt', sep=" ", header=None)

In [101]:
train_df.columns=['patient id', 'filename', 'class', 'data source']

In [102]:
train_df=train_df.drop(['patient id', 'data source'], axis=1 )

In [103]:
train_df.head()

,filename,class
0,ARDSSevere.png,negative
1,acute-respiratory-distress-syndrome-ards-1.jpg,negative
2,acute-respiratory-distress-syndrome-ards.jpg,negative
3,ards-secondary-to-tiger-snake-bite.png,negative
4,pneumocystis-pneumonia-2-PA.png,negative


In [104]:
test_df = pd.read_csv('Capstone covid data/test.txt', sep=" ", header=None)
test_df.columns=['id', 'filename', 'class', 'data source' ]
test_df=test_df.drop(['id', 'data source'], axis=1 )

In [105]:
test_df.head()

,filename,class
0,MIDRC-RICORD-1C-419639-003251-46647-0.png,positive
1,MIDRC-RICORD-1C-419639-001464-39871-0.png,positive
2,MIDRC-RICORD-1C-419639-000918-78965-0.png,positive
3,MIDRC-RICORD-1C-419639-003318-64285-0.png,positive
4,MIDRC-RICORD-1C-419639-001015-81591-0.png,positive


In [106]:
train_df['class'].value_counts()

negative    13793
positive     2158
Name: class, dtype: int64

In [107]:
negative  = train_df[train_df['class']=='negative']  
positive = train_df[train_df['class']=='positive'] 
from sklearn.utils import resample

df_majority_downsampled = resample(negative, replace = True, n_samples = 2158) 

train_df = pd.concat([positive, df_majority_downsampled])

from sklearn.utils import shuffle
train_df = shuffle(train_df) 

In [108]:
train_df['class'].value_counts()

negative    2158
positive    2158
Name: class, dtype: int64

In [109]:
train_df, valid_df = train_test_split(train_df, train_size=0.9, random_state=0)

In [110]:
print(f"Negative and positive values of train: {train_df['class'].value_counts()}")
print(f"Negative and positive values of validation: {valid_df['class'].value_counts()}")
print(f"Negative and positive values of test: {test_df['class'].value_counts()}")

Negative and positive values of train: negative    1945
positive    1939
Name: class, dtype: int64
Negative and positive values of validation: positive    219
negative    213
Name: class, dtype: int64
Negative and positive values of test: negative    200
positive    200
Name: class, dtype: int64


In [111]:
train_path = 'Capstone covid data/train'  #directory path
test_path = 'Capstone covid data/test'

In [112]:
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [113]:
train_gen = train_datagen.flow_from_dataframe(dataframe = train_df, directory=train_path, x_col='filename', 
                                              y_col='class', target_size=(224,224), batch_size=64, 
                                               class_mode='binary')
valid_gen = test_datagen.flow_from_dataframe(dataframe = valid_df, directory=train_path, x_col='filename',
                                             y_col='class', target_size=(224,224), batch_size=64, 
                                            class_mode='binary')
test_gen = test_datagen.flow_from_dataframe(dataframe = test_df, directory=test_path, x_col='filename', 
                                            y_col='class', target_size=(224,224), batch_size=64,
                                             class_mode='binary')


Found 3884 validated image filenames belonging to 2 classes.
Found 432 validated image filenames belonging to 2 classes.
Found 400 validated image filenames belonging to 2 classes.


In [287]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import * 
from keras.preprocessing import image 

In [288]:
CNN=load_model('CNN_model_covid.h5')
RNET=load_model('ResNet_model_covid.h5')
GN=load_model('GoogleNet_final_covid_latest.h5')

In [289]:
members=[CNN, RNET, GN]

In [293]:
def prediction_y(model):
    y_test=[]
    for j in range (len(valid_df.filename)):
        img_path="Capstone covid data/train/"+valid_df.filename.array[j]
        i = image.load_img(img_path, target_size=(224,224))
        i = image.img_to_array(i)/255.0
        i = i.reshape(1, 224,224,3)
        p=model.predict(i)
        if(model == GN):
            flag=0
            if(p[0][0]>=0.5):
                flag+=1
            if(p[1][0]>=0.5):
                flag+=1
            if(p[2][0]>=0.5):
                flag+=1

            if(flag>=2):
                y_test.append(1)
            elif(flag<2):
                y_test.append(0)
        else:
            if(p[0][0]>0.5):
                y_test.append(1)
            else:
                y_test.append(0)
    
    return y_test



In [294]:
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from tensorflow.keras.utils import to_categorical
from numpy import dstack

In [306]:
stackX = None
for model in members:
    yhat=np.array(prediction_y(model))
    if stackX is None:
            
        stackX = yhat
    else:
        stackX = dstack((stackX, yhat))
        
# stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
   


In [322]:
y_actual=[]
y=np.array(valid_df['class'].tolist())

In [323]:
for i in range(len(y)):
    if(y[i]=='negative'):
        y_actual.append(0)
    elif(y[i]=='positive'):
        y_actual.append(1)
    else:
        print('error')
        break

In [324]:
np.shape(stackX)

(432, 3)

In [325]:
stackx1=stackX

In [327]:
# stackX = stackX.reshape((stackX.shape[0]*stackX.shape[1],stackX.shape[2]))

In [328]:
stackX

array([[0, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1]])

In [329]:
y_actual

[0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,


In [335]:
type(stackX[0])

numpy.ndarray

In [339]:
import statistics
Y_PRE=[]
for i in range (len(stackX)):
    demo=list(stackX[i])
    if(demo.count(1)>=2):
        Y_PRE.append(1)
    else:
        Y_PRE.append(0)

In [340]:
Y_PRE

[0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,


In [341]:
np.shape(Y_PRE)

(432,)

In [342]:
np.shape(y_actual)

(432,)

In [345]:
import sklearn
sklearn.metrics.accuracy_score(y_actual, Y_PRE, normalize=True, sample_weight=None)

0.9259259259259259

In [ ]:
def predict_label(img_path):
    i = image.load_img(img_path, target_size=(224,224))
    i = image.img_to_array(i)/255.0
    i = i.reshape(1, 224,224,3)
    Ans=0
    for model in members: 
        p = model.predict(i)
        if(model == GN):
            flag=0
            if(p[0][0]>=0.5):
                flag+=1
            if(p[1][0]>=0.5):
                flag+=1
            if(p[2][0]>=0.5):
                flag+=1
            if(flag>=2):
                Ans+=1
        else:
            if(p[0][0]>0.5):
                Ans+=1
    if(Ans>=2):
        return POSITIVE
    else:
        return NEGATIVE